In [ ]:
import pandas as pd
import numpy as np
import os
import torch
import torch.nn as nn
import torchtext
from torchtext.legacy.data import Field,LabelField,BucketIterator,TabularDataset
import spacy
from torchtext.data.utils import get_tokenizer
from torch.autograd import Variable
import torch.nn.functional as F
import random

In [ ]:
dirs="/content/drive/MyDrive/RNN_Insulting_Comment/"

In [ ]:
data=pd.read_csv("/content/drive/MyDrive/RNN_Insulting_Comment/RNN_train_Insulting.csv")
data.head()

,Insult,Comment
0,1,you fuck your dad
1,0,i really do understand your it seems that you ...
2,0,of canadians can and has been wrong before now...
3,0,listen if you do nt wanna get married to a man...
4,0,txecnh cuxed t giang n t thxec chxfang ta nai ...


In [ ]:
tokenizer=spacy.load('en_core_web_sm')

In [ ]:
TEXT=Field(tokenize ='spacy',tokenizer_language = 'en_core_web_sm',include_lengths=True)
Label=LabelField(dtype=torch.float)

In [ ]:
field=[("label",Label),("text",TEXT)]

In [ ]:
training_data=TabularDataset(path="/content/drive/MyDrive/RNN_Insulting_Comment/RNN_train_Insulting.csv",format="csv",
                             fields=field,skip_header=True)

print(vars(training_data[3]))

{'label': '0', 'text': ['listen', 'if', 'you', 'do', 'nt', 'wanna', 'get', 'married', 'to', 'a', 'man', 'or', 'a', 'women', 'do', 'nt', 'do', 'it', 'what', 'would', 'it', 'bother', 'you', 'if', 'gay', 'people', 'got', 'married', 'stay', 'in', 'your', 'lane', 'do', 'you', 'let', 'them', 'do', 'them', 'and', 'your', 'god', 'is', 'so', 'nice', 'but', 'quick', 'to', 'judg', 'if', 'your', 'not', 'like', 'him', 'thought', 'you', 'was', 'nt', 'suppose', 'to', 'judge', 'people']}


In [ ]:
train_data,valid_data=training_data.split(split_ratio=0.7,random_state=random.seed(2020))

In [ ]:
data.shape,len(train_data),len(valid_data)

((3947, 2), 2763, 1184)

In [ ]:
MAX_VOCAB_SIZE=25_000
TEXT.build_vocab(train_data,max_size=MAX_VOCAB_SIZE,vectors="glove.6B.100d",unk_init=torch.Tensor.normal_)
Label.build_vocab(train_data)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
train_iter,valid_iter=BucketIterator.splits((train_data,valid_data),
                                     sort_key=lambda x:len(x.text),
                                     batch_size=32,
                                     sort_within_batch=True,
                                     device=device)

In [ ]:
for i in train_iter:
  print(i.text[0][0])
  break

tensor([  19,   58, 7459, 3378,    3, 6899,  267,  287,    8,    8,  174,    3,
           8,  931,  104,    8, 5102,    6,  120,    3, 5869,   10,    3, 4789,
         122,    5,    3,   21,  716,   63,   58,    8])


In [ ]:
TEXT.vocab.stoi[TEXT.pad_token]

1

In [ ]:
class LSTM(nn.Module):
  def __init__(self,vocab_size,embedding_dim,hidden_dim,n_layer,output_dim,pad_token,dropout):
    super(LSTM,self).__init__()

    self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_token)
    #LSTM Layer
    self.lstm=nn.LSTM(embedding_dim,hidden_dim,num_layers=n_layer,bidirectional=True,dropout=dropout)
    #linear layer
    self.fc=nn.Linear(hidden_dim*2,output_dim)
    #dropout
    self.dropout=nn.Dropout(dropout)
    # self.sigmoid=nn.Sigmoid()

  def forward(self,text,text_len):
    #Forward pass into the embedding layer
    for i, _ in enumerate(text_len):
        if text_len[i] == 0:
          text_len[i] = 1
    embedd=self.dropout(self.embedding(text))
    pack_output=nn.utils.rnn.pack_padded_sequence(embedd,text_len.to("cpu"))
    #Now We Apply lstm and it's return output and a tuple of final hidden state and final cell state
    packed_output,(hidden,cell)=self.lstm(pack_output)
    #Unpack sequence
    output,output_lengths=nn.utils.rnn.pad_packed_sequence(packed_output)
    #Forward pass and backward pass
    hidden =self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
    # hidden=hidden[:, -1, :]
    # hidden=torch.tanh(hidden)


    fc=self.fc(hidden)

    # fc2=self.linear2(fc)

    # dense_output=self.sigmoid(fc2)
    

    return fc


In [ ]:
vocab_size=len(TEXT.vocab)
embedding_dim=100
hidden_dim=256
n_layers=2
output_dim=1
dropout=0.5
pad_token=TEXT.vocab.stoi[TEXT.pad_token]
model=LSTM(vocab_size,embedding_dim,hidden_dim,n_layers,output_dim,pad_token,dropout)
model

LSTM(
  (embedding): Embedding(11814, 100, padding_idx=1)
  (lstm): LSTM(100, 256, num_layers=2, dropout=0.5, bidirectional=True)
  (fc): Linear(in_features=512, out_features=1, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [ ]:
for data in train_iter:
  text,text_len=data.text
  print(model(text,text_len))


tensor([[0.4870],
        [0.4830],
        [0.4712],
        [0.4905],
        [0.4960],
        [0.4830],
        [0.5039],
        [0.4760],
        [0.5184],
        [0.4659],
        [0.4953],
        [0.5154],
        [0.4806],
        [0.4830],
        [0.4961],
        [0.4964],
        [0.4913],
        [0.4924],
        [0.4751],
        [0.4842],
        [0.4902],
        [0.4864],
        [0.4907],
        [0.4739],
        [0.4854],
        [0.4810],
        [0.4951],
        [0.4691],
        [0.4871],
        [0.4939],
        [0.5103],
        [0.4812]], grad_fn=<SigmoidBackward>)
tensor([[0.4911],
        [0.4976],
        [0.4875],
        [0.4817],
        [0.4928],
        [0.4994],
        [0.4888],
        [0.4906],
        [0.4907],
        [0.4852],
        [0.5120],
        [0.5057],
        [0.4960],
        [0.4880],
        [0.4780],
        [0.4907],
        [0.4896],
        [0.5031],
        [0.5018],
        [0.5046],
        [0.4913],
        [0.5011],


In [ ]:
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

torch.Size([11814, 100])


In [ ]:
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[ 1.1378,  1.4964,  0.2838,  ...,  1.0049,  0.3367, -0.2661],
        [-2.3730, -0.2436,  0.7924,  ...,  0.2624, -0.5812, -0.4321],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.0205, -0.4512,  0.6993,  ...,  0.0381,  0.2786,  0.2889],
        [ 0.7834, -0.5083, -0.9474,  ...,  0.5490, -0.6405,  1.1772],
        [ 0.8529, -0.3760, -0.8629,  ...,  0.3011, -0.9170,  0.3196]])

In [ ]:

UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(embedding_dim)
model.embedding.weight.data[pad_token] = torch.zeros(embedding_dim)

print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.0205, -0.4512,  0.6993,  ...,  0.0381,  0.2786,  0.2889],
        [ 0.7834, -0.5083, -0.9474,  ...,  0.5490, -0.6405,  1.1772],
        [ 0.8529, -0.3760, -0.8629,  ...,  0.3011, -0.9170,  0.3196]])


In [ ]:
criterion=nn.BCEWithLogitsLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=0.001)

In [ ]:
def binary_accuracy(preds,y):
  rounded_preds=torch.round(torch.sigmoid(preds))
  correct=(rounded_preds == y).float()
  acc=correct.sum() / len(correct)
  return acc

In [ ]:
def train(model, iterator, optimizer, criterion):
    
    #initialize every epoch 
    epoch_loss = 0
    epoch_acc = 0
    
    #set the model in training phase
    model.train()  
    
    for batch in iterator:
        
        #resets the gradients after every batch
        optimizer.zero_grad()   
        
        #retrieve text and no. of words
        text, text_lengths = batch.text
        label=batch.label
        #convert to 1D tensor
        predictions = model(text, text_lengths).squeeze(1)  
        
        #compute the loss
        loss = criterion(predictions, batch.label)        
        
        #compute the binary accuracy
        acc = binary_accuracy(predictions,batch.label)   
        
        #backpropage the loss and compute the gradients
        loss.backward()       
        
        #update the weights
        optimizer.step()      
        
        #loss and accuracy
        epoch_loss += loss.item()  
        epoch_acc += acc.item()

        # lr_schedule.step(epoch_loss)    
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def evaluate(model,iterator,criterion):
  #initialize every epoch
    epoch_loss = 0
    epoch_acc = 0

    #deactivating dropout layers
    model.eval()
    
    #deactivates autograd
    with torch.no_grad():
    
        for batch in iterator:
          
            #retrieve text and no. of words
            text, text_lengths = batch.text
            label=batch.label
            #convert to 1d tensor
            predictions = model(text, text_lengths).squeeze(1)
            
            #compute loss and accuracy
            loss = criterion(predictions,batch.label)
            acc = binary_accuracy(predictions, batch.label)
            
            #keep track of loss and accuracy
            epoch_loss += loss.item()
            epoch_acc += acc.item()
            # lr_schedule.step(epoch_loss)
            
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
N_EPOCHS = 10
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
     
    #train the model
    train_loss, train_acc = train(model, train_iter, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iter, criterion)
    
    #save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), dirs+'saved_weights_1.pt')
    
    
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

	Train Loss: 0.522 | Train Acc: 75.43%
	 Val. Loss: 0.466 |  Val. Acc: 77.20%
	Train Loss: 0.446 | Train Acc: 78.85%
	 Val. Loss: 0.542 |  Val. Acc: 78.29%
	Train Loss: 0.432 | Train Acc: 80.15%
	 Val. Loss: 0.401 |  Val. Acc: 82.26%
	Train Loss: 0.354 | Train Acc: 84.41%
	 Val. Loss: 0.361 |  Val. Acc: 84.88%
	Train Loss: 0.322 | Train Acc: 85.92%
	 Val. Loss: 0.362 |  Val. Acc: 84.29%
	Train Loss: 0.292 | Train Acc: 87.54%
	 Val. Loss: 0.348 |  Val. Acc: 84.71%
	Train Loss: 0.265 | Train Acc: 89.23%
	 Val. Loss: 0.352 |  Val. Acc: 83.78%
	Train Loss: 0.227 | Train Acc: 90.80%
	 Val. Loss: 0.382 |  Val. Acc: 82.94%
	Train Loss: 0.204 | Train Acc: 91.63%
	 Val. Loss: 0.415 |  Val. Acc: 84.71%
	Train Loss: 0.171 | Train Acc: 93.53%
	 Val. Loss: 0.380 |  Val. Acc: 84.54%


In [ ]:
# accuracy plots
# plt.figure(figsize=(10, 7))
# plt.plot(train_accuracy, color='green', label='train accuracy')
# plt.plot(val_accuracy, color='blue', label='validataion accuracy')
# plt.xlabel('Epochs')
# plt.ylabel('Accuracy')
# plt.legend()
# plt.savefig(dirs+'accuracy.png')
# plt.show()
# # loss plots
# plt.figure(figsize=(10, 7))
# plt.plot(train_loss, color='orange', label='train loss')
# plt.plot(val_loss, color='red', label='validataion loss')
# plt.xlabel('Epochs')
# plt.ylabel('Loss')
# plt.legend()
# plt.savefig(dirs+'loss.png')
# plt.show()
    
# # serialize the model to disk
# # print('Saving model...')
# # torch.save(model.state_dict(), args['model'])
 
# print('TRAINING COMPLETE')

In [ ]:
model.load_state_dict(torch.load(dirs+'saved_weights_1.pt'))

test_loss, test_acc = evaluate(model, valid_iter, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.348 | Test Acc: 84.71%


In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')

def predict_sentiment(model, sentence):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)
    prediction = torch.sigmoid(model(tensor, length_tensor))
    return prediction.item()

In [ ]:
predict_sentiment(model,"txecnh cuxed t giang n t thxec chxfang ta nai yxeau bxecnh khi bxecnh sau tranh con thxeam")

0.08382263779640198

In [ ]:
df=pd.read_csv("/content/drive/MyDrive/RNN_Insulting_Comment/RNN_train_Insulting.csv")
df.head()

,Insult,Comment
0,1,you fuck your dad
1,0,i really do understand your it seems that you ...
2,0,of canadians can and has been wrong before now...
3,0,listen if you do nt wanna get married to a man...
4,0,txecnh cuxed t giang n t thxec chxfang ta nai ...


In [ ]:
df["Comment"][4]

'txecnh cuxed t giang n t thxec chxfang ta nai yxeau bxecnh khi bxecnh sau tranh con thxeam'